In [1]:
%%capture
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from keras.utils import np_utils
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import SGD
from time import time
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.callbacks import Callback

import numpy as np
import pandas as pd
import matplotlib, os, math

os.environ["CUDA_VISIBLE_DEVICES"]="1"

### 1.Prepare data

In [2]:
from time import time
import numpy as np

start_time = time()

print('Loading X_train...')
X_train = np.load('/home/zxt/data/21_category/224/5k/X_train.npy')


end_time = time()
print('Running time: %s seconds'%(end_time - start_time))

Loading X_train...
Running time: 909.4823398590088 seconds


In [3]:
len(X_train)

94500

In [4]:
from time import time
import numpy as np

start_time = time()

print('Loading X_validate...')
X_validate = np.load('/home/zxt/data/21_category/224/5k/X_validate.npy')

print('Loading X_test...')
X_test = np.load('/home/zxt/data/21_category/224/5k/X_test.npy')

print('Loading labels...')
Y_train = np.load('/home/zxt/data/21_category/224/5k/Y_train.npy')
Y_validate = np.load('/home/zxt/data/21_category/224/5k/Y_validate.npy')
Y_test = np.load('/home/zxt/data/21_category/224/5k/Y_test.npy')

end_time = time()
print('Running time: %s seconds'%(end_time - start_time))

Loading X_validate...
Loading X_test...
Loading labels...
Running time: 100.68361759185791 seconds


In [ ]:
# X_validate = np.vstack((X_validate_part1, X_validate_part2))

In [5]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_validate.shape[0], 'validate samples')
print(X_test.shape[0], 'test samples')

X_train shape: (94500, 224, 224, 3)
94500 train samples
5250 validate samples
5250 test samples


In [6]:
X_train /=255.
X_validate /=  255.
X_test /= 255.

In [11]:
# Y_train label
for i in Y_train:
    if i[0]=='elephant':
        i[0]=0
    if i[0]=='wild_boar':
        i[0]=1
    if i[0]=='gazellethomosons':
        i[0]=2
    if i[0]=='squirrel':
        i[0]=3 
    if i[0]=='hedgehog':
        i[0]=4
    if i[0]=='guineafowl':
        i[0]=5
    if i[0]=='moose':
        i[0]=6
    if i[0]=='coyote':
        i[0]=7   
    if i[0]=='bird':
        i[0]=8
    if i[0]=='fox':
        i[0]=9
    if i[0]=='giraffe':
        i[0]=10
    if i[0]=='buffalo':
        i[0]=11
    if i[0]=='hare':
        i[0]=12
    if i[0]=='vehicle':
        i[0]=13
    if i[0]=='wildebeest':
        i[0]=14
    if i[0]=='cattle':
        i[0]=15 
    if i[0]=='skunk':
        i[0]=16
    if i[0]=='lion':
        i[0]=17
    if i[0]=='zebra':
        i[0]=18
    if i[0]=='black_bear':
        i[0]=19
    if i[0]=='racoon':
        i[0]=20  

In [12]:
for i in Y_validate:
    if i[0]=='elephant':
        i[0]=0
    if i[0]=='wild_boar':
        i[0]=1
    if i[0]=='gazellethomosons':
        i[0]=2
    if i[0]=='squirrel':
        i[0]=3 
    if i[0]=='hedgehog':
        i[0]=4
    if i[0]=='guineafowl':
        i[0]=5
    if i[0]=='moose':
        i[0]=6
    if i[0]=='coyote':
        i[0]=7   
    if i[0]=='bird':
        i[0]=8
    if i[0]=='fox':
        i[0]=9
    if i[0]=='giraffe':
        i[0]=10
    if i[0]=='buffalo':
        i[0]=11
    if i[0]=='hare':
        i[0]=12
    if i[0]=='vehicle':
        i[0]=13
    if i[0]=='wildebeest':
        i[0]=14
    if i[0]=='cattle':
        i[0]=15 
    if i[0]=='skunk':
        i[0]=16
    if i[0]=='lion':
        i[0]=17
    if i[0]=='zebra':
        i[0]=18
    if i[0]=='black_bear':
        i[0]=19
    if i[0]=='racoon':
        i[0]=20   

In [13]:
for i in Y_test:
    if i[0]=='elephant':
        i[0]=0
    if i[0]=='wild_boar':
        i[0]=1
    if i[0]=='gazellethomosons':
        i[0]=2
    if i[0]=='squirrel':
        i[0]=3 
    if i[0]=='hedgehog':
        i[0]=4
    if i[0]=='guineafowl':
        i[0]=5
    if i[0]=='moose':
        i[0]=6
    if i[0]=='coyote':
        i[0]=7   
    if i[0]=='bird':
        i[0]=8
    if i[0]=='fox':
        i[0]=9
    if i[0]=='giraffe':
        i[0]=10
    if i[0]=='buffalo':
        i[0]=11
    if i[0]=='hare':
        i[0]=12
    if i[0]=='vehicle':
        i[0]=13
    if i[0]=='wildebeest':
        i[0]=14
    if i[0]=='cattle':
        i[0]=15 
    if i[0]=='skunk':
        i[0]=16
    if i[0]=='lion':
        i[0]=17
    if i[0]=='zebra':
        i[0]=18
    if i[0]=='black_bear':
        i[0]=19
    if i[0]=='racoon':
        i[0]=20 

In [14]:
# one-hot encoder[only accpet int]
y_train=np_utils.to_categorical(Y_train,21)
y_validate=np_utils.to_categorical(Y_validate,21)
y_test=np_utils.to_categorical(Y_test,21)

In [15]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### 2.Model structure

In [16]:
def alexnet(input_shape, n_classes):
    input = Input(input_shape)
  
    # actually batch normalization didn't exist back then
    # they used LRN (Local Response Normalization) for regularization
    x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Conv2D(256, 5, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
    x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
  
    output = Dense(n_classes, activation='softmax')(x)
  
    model = Model(input, output)
    return model

In [17]:
# prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

In [18]:
from keras.callbacks import Callback
class Metrics(Callback):

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, y_val = self.validation_data[:2]
        y_pred = self.model.predict(X_val)

        y_pred_cat = keras.utils.to_categorical(
            y_pred.argmax(axis=1),
            num_classes=num_classes
        )

        _val_f1 = f1_score(y_val, y_pred_cat, average='macro')
        _val_recall = recall_score(y_val, y_pred_cat, average='macro')
        _val_precision = precision_score(y_val, y_pred_cat, average='macro')

        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)

        print((f"val_f1: {_val_f1:.4f}"
               f" - val_precision: {_val_precision:.4f}"
               f" - val_recall: {_val_recall:.4f}"))

        return

### 3.Modeling

In [19]:
num_train_samples = len(Y_train)
num_val_samples = len(Y_validate)
batch_size = 128
epochs = 100
n_classes = 21
input_shape = 224,224,3

In [20]:
K.clear_session()
model = alexnet(input_shape, n_classes)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0   

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [24]:
num_classes = 21
f1_metrics = Metrics()

In [25]:
hist = model.fit(
    X_train, 
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[early_stopping,f1_metrics],
    validation_data = (X_validate,y_validate))

Train on 94500 samples, validate on 5250 samples
Epoch 1/100
94500/94500 [==============================] - 449s 5ms/step - loss: 1.3692 - accuracy: 0.5443 - val_loss: 1.5844 - val_accuracy: 0.4771
val_f1: 0.4514 - val_precision: 0.6050 - val_recall: 0.4747
Epoch 2/100
94500/94500 [==============================] - 425s 5ms/step - loss: 0.8585 - accuracy: 0.7061 - val_loss: 0.9755 - val_accuracy: 0.6665
val_f1: 0.6712 - val_precision: 0.7089 - val_recall: 0.6689
Epoch 3/100
94500/94500 [==============================] - 426s 5ms/step - loss: 0.6546 - accuracy: 0.7754 - val_loss: 1.8290 - val_accuracy: 0.4164
val_f1: 0.4115 - val_precision: 0.6519 - val_recall: 0.4154
Epoch 4/100
94500/94500 [==============================] - 426s 5ms/step - loss: 0.5094 - accuracy: 0.8258 - val_loss: 1.2066 - val_accuracy: 0.5674
val_f1: 0.5638 - val_precision: 0.6683 - val_recall: 0.5730
Epoch 5/100
94500/94500 [==============================] - 426s 5ms/step - loss: 0.3944 - accuracy: 0.8662 - val_lo

KeyboardInterrupt: 

In [30]:
# parallel_model.save('/home/zxt/data/model/alex_basic_5k_224.h5')

In [26]:
model.save('/home/zxt/data/model/21category/alex_basic_5k_224_model.h5')

### 4.Evaluation

In [27]:
history_df = pd.DataFrame(hist.history)
history_df[['loss', 'val_loss']].plot()
history_df[['accuracy', 'val_accuracy']].plot()

NameError: name 'hist' is not defined

### 5.Prediction on X_test dataset

In [28]:
loss,accuracy = model.evaluate(X_test,y_test)
print('\ntest loss',loss)
print('accuracy',accuracy)

5250/5250 [==============================] - 11s 2ms/step

test loss 0.489455084386326
accuracy 0.9074285626411438


In [29]:
y_pred = model.predict(X_test,batch_size=1)

In [30]:
y_pred

array([[4.8132285e-09, 1.7941906e-08, 5.1421116e-12, ..., 1.4136230e-09,
        7.8386717e-13, 1.0653484e-09],
       [7.1362444e-11, 2.3809670e-09, 2.9026877e-15, ..., 3.0008346e-10,
        4.8786825e-10, 2.7714447e-11],
       [7.0330959e-12, 3.5752894e-09, 7.4230934e-11, ..., 8.0918087e-14,
        1.0880810e-09, 2.8406225e-10],
       ...,
       [1.0683399e-10, 1.3301432e-07, 9.9817257e-09, ..., 6.8673046e-11,
        2.0485328e-14, 6.6858076e-05],
       [1.9701302e-08, 1.0886635e-11, 7.8831018e-11, ..., 1.0174849e-14,
        3.9884657e-08, 2.4080984e-14],
       [8.9193620e-08, 3.3112660e-03, 1.2191908e-08, ..., 1.4292359e-08,
        1.1597029e-07, 2.0739940e-01]], dtype=float32)

### 6.Check each categoryaccuracy

In [31]:
a = list(y_pred)
y_prediction=[]
for i in a:
    j =list(i)
    y_prediction.append(j.index(max(j)))

In [32]:
y_prediction[:5]

[13, 7, 12, 2, 0]

In [33]:
Y_test[:5]

array([['13'],
       ['7'],
       ['12'],
       ['2'],
       ['0']], dtype='<U16')

In [34]:
# array to dataframe
y_real=pd.DataFrame(Y_test,columns=['label'])

In [35]:
y_real.head(2)

,label
0,13
1,7


In [36]:
y_predict = pd.DataFrame(y_prediction,columns=['label_pre'])

In [37]:

y_predict.head(2)

,label_pre
0,13
1,7


In [38]:
prediction_df = pd.concat([y_real,y_predict],axis=1)
prediction_df['label']=prediction_df['label'].astype(int)
prediction_df['diff']=prediction_df['label']-prediction_df['label_pre']


In [39]:
prediction_df

,label,label_pre,diff
0,13,13,0
1,7,7,0
2,12,12,0
3,2,2,0
4,0,0,0
...,...,...,...
5245,12,12,0
5246,11,11,0
5247,15,15,0
5248,13,13,0


In [71]:
prediction_df.to_csv(r'/home/zxt/data/results/prediction_df.csv',index=False)

In [40]:
onehot_keylist = ['elephant','wild_boar','gazellethomosons','squirrel','hedgehog','guineafowl','moose','coyote','bird',
                  'fox','giraffe','buffalo','hare','vehicle','wildebeest','cattle','skunk','lion','zebra','black_bear',
                  'racoon']
onehot_dict = {'elephant':0,'wild_boar':1,'gazellethomosons':2,'squirrel':3,'hedgehog':4,'guineafowl':5,'moose':6,'coyote':7,
              'bird':8,'fox':9,'giraffe':10,'buffalo':11,'hare':12,'vehicle':13,'wildebeest':14,'cattle':15,'skunk':16,'lion':17,
              'zebra':18,'black_bear':19,'racoon':20}

for animal in onehot_keylist:
    name = animal
    animal = prediction_df[prediction_df['label']== onehot_dict[animal]]
    acc = animal.loc[:,'diff'].value_counts()[0]/len(animal)
    print("%s acc: %10.3f" %(name,acc) )

elephant acc:      0.849
wild_boar acc:      0.826
gazellethomosons acc:      0.945
squirrel acc:      0.943
hedgehog acc:      0.996
guineafowl acc:      0.968
moose acc:      0.989
coyote acc:      0.905
bird acc:      0.886
fox acc:      0.915
giraffe acc:      0.944
buffalo acc:      0.883
hare acc:      0.973
vehicle acc:      0.950
wildebeest acc:      0.789
cattle acc:      0.740
skunk acc:      0.949
lion acc:      0.876
zebra acc:      0.833
black_bear acc:      0.891
racoon acc:      0.961
